# Embedding
OpenAI의 텍스트 임베딩은 텍스트 문자열의 관련성을 측정합니다. 임베딩은 일반적으로 다음 용도로 사용됩니다.

- 두 텍스트 사이의 관련성을 측정하는 데 사용할 수 있는 텍스트의 숫자 표현  
- 검색, 클러스터링, 추천, 이상 탐지 및 분류 작업에 유용

    - 검색 (쿼리 문자열과의 관련성을 기준으로 결과 순위 지정)  
    - 클러스터링 (텍스트 문자열이 유사성을 기준으로 그룹화)  
    - 추천 (관련 문자열이 포함된 항목을 추천)  
    - 이상 탐지 (관련성이 거의 없는 이상값이 식별되는 경우)  
    - 다양성 측정 (유사성 분포를 분석)
    - 분류 (텍스트 문자열이 가장 유사한 레이블로 분류)
  
임베딩은 부동 소수점 숫자의 벡터(목록)입니다. 두 벡터 사이의 거리는 관련성 을 측정합니다. 거리가 작을수록 관련성이 높음을 나타내고, 거리가 멀면 관련성이 낮다는 것을 나타냅니다.  

<img src=https://cdn.sanity.io/images/vr8gru94/production/e016bbd4d7d57ff27e261adf1e254d2d3c609aac-2447x849.png width=600 />

### 벡터 유사도:

이제 두 개의 서로 다른 영화에 대한 두 개의 숫자 목록이 있다고 가정해 보겠습니다. 영화가 비슷한지 어떻게 알 수 있나요? 여기서 벡터 유사도가 측정됩니다.

요약하면,

- **임베딩**은 컴퓨터가 이해할 수 있는 특수 숫자 코드로 영화 등의 자세한 설명을 작성하는 것과 같습니다.
- **벡터 유사도**는 두 영화와 같이 두 숫자 집합이 나타내는 항목이 서로 얼마나 유사한지 알 수 있습니다.


<img src=https://cdn.sanity.io/images/vr8gru94/production/5a5ba7e0971f7b6dc4697732fa8adc59a46b6d8d-338x357.png width=200 />

$\cos(\theta) = \frac{A \cdot B}{||A|| \times ||B||}$

In [1]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())      # read local .env file

True

In [2]:
from openai import OpenAI 
client = OpenAI()  # OpenAI 클라이언트 생성

Model = "gpt-4o-mini"
embedding_model = "text-embedding-3-small"  # 사용할 텍스트 임베딩 모델

## API 사용 방법

- 기본적으로 임베딩 벡터의 길이는 text-embedding-3-small의 경우 1536이고 text-embedding-3-large의 경우 3072입니다. 차원 매개변수를 전달하면 임베딩이 개념을 나타내는 속성을 잃지 않고  임베딩의 차원을 줄일 수 있습니다.

In [3]:
# 텍스트 임베딩 생성 요청
response = client.embeddings.create(
    input="안녕하세요. 건강하십시요.",  # 임베딩을 생성할 텍스트 입력
    model=embedding_model  # 사용할 모델 설정
)

# 응답에서 임베딩 데이터의 처음 10개 요소를 출력
print(len(response.data[0].embedding))
print()
print(response.data[0].embedding[:10])

1536

[-0.014604989439249039, -0.042497579008340836, -0.004620332270860672, 0.03440370410680771, 0.01206499058753252, -0.029304753988981247, -0.053529512137174606, 0.012083945795893669, 0.005497010890394449, -0.04973847046494484]


In [4]:
import numpy as np

def get_embedding(text: str, model=embedding_model):
    text = text.replace("\n", " ")  # 성능에 부정적 영향을 줄 수 있는 \n 제거
    response = client.embeddings.create(input=[text], model=model)
    return response.data[0].embedding

# 코사인 유사도 계산 함수
def cosine_similarity(vec1, vec2):
    dot_product = np.dot(vec1, vec2)  # 벡터 내적
    norm_vec1 = np.linalg.norm(vec1)  # 벡터 1의 크기 (L2 Norm)
    norm_vec2 = np.linalg.norm(vec2)  # 벡터 2의 크기 (L2 Norm)
    return dot_product / (norm_vec1 * norm_vec2)  # 코사인 유사도 공식 적용

In [6]:
texts = [
     "음식이 맛있었고 웨이터가 매우 친절했습니다.",  # 리뷰 0
     "분위기는 좋았지만 음식은 그냥 그래요.",         # 리뷰 1
     "저는 음식이 마음에 들었고 웨이터는 매우 세심했습니다."  # 리뷰 2
]

# 각 텍스트를 벡터(임베딩)로 변환하여 리스트에 저장
embeddings = [get_embedding(text) for text in texts]

# 두 개의 벡터(임베딩) 사이의 코사인 유사도를 계산하여 비교
simi_0_1 = cosine_similarity(embeddings[0], embeddings[1])  # 리뷰 0과 1 비교
simi_0_2 = cosine_similarity(embeddings[0], embeddings[2])  # 리뷰 0과 2 비교
simi_1_2 = cosine_similarity(embeddings[1], embeddings[2])  # 리뷰 1과 2 비교

# 각 텍스트 출력 (인덱스와 함께 표시)
for i, text in enumerate(texts):
    print(f"test {i} : {text}")

# 코사인 유사도 출력
print(f"텍스트 0과 텍스트 1 사이의 유사도: {simi_0_1:.2f}")
print(f"텍스트 0과 텍스트 2 사이의 유사도: {simi_0_2:.2f}")
print(f"텍스트 1과 텍스트 2 사이의 유사도: {simi_1_2:.2f}")

test 0 : 음식이 맛있었고 웨이터가 매우 친절했습니다.
test 1 : 분위기는 좋았지만 음식은 그냥 그래요.
test 2 : 저는 음식이 마음에 들었고 웨이터는 매우 세심했습니다.
텍스트 0과 텍스트 1 사이의 유사도: 0.38
텍스트 0과 텍스트 2 사이의 유사도: 0.78
텍스트 1과 텍스트 2 사이의 유사도: 0.42


## Amazon 고급 음식 리뷰를 이용한 Text Search

데이터 세트에는 2012년 10월까지 Amazon 사용자가 남긴 총 568,454개의 음식 리뷰가 포함되어 있습니다. 이 중 가장 최근 리뷰 1,000개로 구성된 이 데이터 세트의 하위 세트를 사용합니다. 리뷰는 영어로 작성되며 긍정적이거나 부정적입니다. 각 리뷰에는 ProductId, UserId, 점수, 리뷰 제목(요약) 및 리뷰 본문(텍스트)이 있습니다.

리뷰 요약과 리뷰 텍스트를 하나의 결합 텍스트로 결합합니다. 모델은 이 결합된 텍스트를 인코딩하고 단일 벡터 임베딩을 출력합니다. 

이 예제에서는 api 비용 절약을 위해 1000 개의 한국어 번역 버전을 사용합니다.

In [7]:
import pandas as pd

# 영어 원본 데이터셋: "data/fine_food_reviews_1k.csv"
# 한국어로 번역된 데이터셋: "data/fine_food_reviews_1k_fully_translated_korean.csv"
input_datapath = "data/fine_food_reviews_1k_fully_translated_korean.csv"

# CSV 파일을 데이터프레임으로 읽기 (첫 번째 열을 인덱스로 사용)
df = pd.read_csv(input_datapath, index_col=0)

# 결측값(NaN) 제거
df = df.dropna()

# "Summary"와 "Text"를 결합하여 "combined" 컬럼 생성
df["combined"] = (
    "Title: " + df.Summary.str.strip() + "; Content: " + df.Text.str.strip()
)

# "Time" 컬럼 제거 (분석에 필요하지 않다고 가정)
df.drop("Time", axis=1, inplace=True)

print(df.shape)

df.head(2)

(999, 6)


,ProductId,UserId,Score,Summary,Text,combined
0,B003XPF9BO,A3R7JR3FMEBXQB,5,이런 식으로 어디서 시작하고 멈추는가,시카고 가족에게 가져 오기 위해 일부를 절약하고 싶었지만 노스 캐롤라이나 가족은 4...,Title: 이런 식으로 어디서 시작하고 멈추는가; Content: 시카고 가족에게...
1,B003JK537S,A3JBPC3WFUT5ZP,1,조각으로 도착했습니다,상자를 열었을 때 전혀 기뻐하지 않습니다. 대부분의 고리가 조각으로 부러졌습니다.,Title: 조각으로 도착했습니다; Content: 상자를 열었을 때 전혀 기뻐하지...


In [7]:
#!pip install tiktoken

In [8]:
import tiktoken  # OpenAI의 텍스트 토큰화 라이브러리 임포트

# "cl100k_base"는 OpenAI의 최신 임베딩 모델(`text-embedding-3-small`, `text-embedding-3-large` 등)과 호환되는 인코딩 방식
embedding_encoding = "cl100k_base"

# 지정된 인코딩 방식을 가져와서 사용
encoding = tiktoken.get_encoding(embedding_encoding)

# 텍스트를 토큰화하여 출력
print(encoding.encode("이것은 한권의 책입니다."))

[13094, 28740, 225, 34804, 62398, 166, 114, 234, 21028, 3396, 109, 227, 80052, 13]


In [9]:
# text-embedding-3-small 모델의 최대 입력 토큰 수는 8191이므로, 안전하게 8000으로 설정
max_tokens = 8000  

# 각 리뷰의 토큰 개수 계산
df["n_tokens"] = df.combined.apply(lambda x: len(encoding.encode(x)))

# 최대 토큰 수를 초과하는 리뷰 제거
df = df[df.n_tokens <= max_tokens]

print(df.shape)

df.head(2)

(999, 7)


,ProductId,UserId,Score,Summary,Text,combined,n_tokens
0,B003XPF9BO,A3R7JR3FMEBXQB,5,이런 식으로 어디서 시작하고 멈추는가,시카고 가족에게 가져 오기 위해 일부를 절약하고 싶었지만 노스 캐롤라이나 가족은 4...,Title: 이런 식으로 어디서 시작하고 멈추는가; Content: 시카고 가족에게...,87
1,B003JK537S,A3JBPC3WFUT5ZP,1,조각으로 도착했습니다,상자를 열었을 때 전혀 기뻐하지 않습니다. 대부분의 고리가 조각으로 부러졌습니다.,Title: 조각으로 도착했습니다; Content: 상자를 열었을 때 전혀 기뻐하지...,55


In [8]:
%%time
# 수분(5-6분) 정도 소요됩니다. 

# # "combined" 컬럼(리뷰 제목 + 본문)에 대해 임베딩을 생성하여 "embedding" 컬럼에 저장
# df["embedding"] = df.combined.apply(lambda x: get_embedding(x, model=embedding_model))
# # 임베딩이 포함된 데이터프레임을 CSV 파일로 저장
# df.to_csv("output/fine_food_reviews_with_embeddings_1k.csv")

# 한번 저장해 놓은 임베딩을 시간과 비용 절약을 위해 재사용.
df = pd.read_csv("output/fine_food_reviews_with_embeddings_1k.csv", index_col=0)

# 저장된 임베딩을 리스트 형태로 변환하여 다시 활용 가능하도록 변경
df['embedding'] = df.embedding.apply(eval).to_list()

df.head(2)

CPU times: total: 3.25 s
Wall time: 3.27 s


,ProductId,UserId,Score,Summary,Text,combined,n_tokens,embedding
0,B003XPF9BO,A3R7JR3FMEBXQB,5,이런 식으로 어디서 시작하고 멈추는가,시카고 가족에게 가져 오기 위해 일부를 절약하고 싶었지만 노스 캐롤라이나 가족은 4...,Title: 이런 식으로 어디서 시작하고 멈추는가; Content: 시카고 가족에게...,87,"[-0.0022805905900895596, 0.0007801640313118696..."
1,B003JK537S,A3JBPC3WFUT5ZP,1,조각으로 도착했습니다,상자를 열었을 때 전혀 기뻐하지 않습니다. 대부분의 고리가 조각으로 부러졌습니다.,Title: 조각으로 도착했습니다; Content: 상자를 열었을 때 전혀 기뻐하지...,55,"[0.05709123983979225, 0.005073584616184235, -0..."


- 가장 관련성이 높은 문서를 검색하기 위해 쿼리의 임베딩 벡터와 각 문서 간의 코사인 유사성을 사용하고 가장 높은 점수를 받은 문서를 반환합니다.
- '맛있는 콩' 이라는 제품 설명과 유사한 상위 3개의 리뷰를 검색하여 반환합니다.

In [9]:
product_description = '맛있는 콩'   # 제품 설명

# 주어진 제품 설명에 대한 임베딩 생성
embedding = get_embedding(product_description, model=embedding_model)
len(embedding)

1536

In [10]:
pd.set_option('display.max_colwidth', None)

# 데이터프레임의 각 임베딩과 제품 설명 임베딩 간의 유사도 계산
df['similarities'] = df.embedding.apply(lambda x: cosine_similarity(x, embedding))

# 유사도를 기준으로 내림차순 정렬하고 상위 n개의 리뷰 선택
df.sort_values('similarities', ascending=False).head(3)[['Summary', 'Text']]

,Summary,Text
771,자메이카 푸른 콩,"로스팅하기위한 우수한 커피 콩 우리 가족은 더 많은 양의 맛을 위해 5 파운드를 더 구입했습니다. 짙은 갈색 콩에 구워 질 때, 기름이 콩 자체에 나타나기 전에 455f 17 분"
88,맛있는,나는이 흰 콩 조미료를 즐긴다 그것은 콩에 풍부한 맛을줍니다. 나는 법에있는 어머니 가이 Zatarains 브랜드에 대해 알지 못했고 이제 그녀는 다른 조미료를 트레이하고 있고 그녀는 지금 내가 할 수 없기 때문에 아마존에 대해 매우 좋아합니다.상점에서 찾아서 가족이 원할 때마다 만들 수 있기 때문에이 12 개의 상자를 갖고 싶어
460,아주 맛있는,아주 좋은 것은 여전히 ​​약간 왼쪽을 가지고 있습니다. 나는 짠 맛이 마음에 든다.
